In [2]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist as mnist
import numpy as np

In [4]:
mnist_data = mnist.input_data.read_data_sets("/tmp/data", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
def reshape(x):
    s = tf.reshape(x, [-1, 28, 28, 1])
    d = tf.zeros([x.get_shape().as_list()[0], 28, 27, 1], dtype=tf.float32)
    return tf.concat(2,[d, s])
learning_rate = 0.001
training_iters = 2
batch_size = 128
display_step = 10

X = tf.placeholder(tf.float32,[batch_size, 784])
X_reshaped = reshape(X)
Y = tf.placeholder(tf.float32,[batch_size, 10])

In [ ]:
weights = {
    "W_z" : tf.Variable(tf.random_normal([28, 28, 1, 20])),
    "W_f" : tf.Variable(tf.random_normal([28, 28, 1, 20])),
    "W_o" : tf.Variable(tf.random_normal([28, 28, 1, 20])),
    "W" : tf.Variable(tf.random_normal([400, 10])),
    "h" : tf.Variable(tf.zeros([28, 20, 1], dtype = tf.float32)),
    "c" : tf.Variable(tf.zeros([28, 20, 1], dtype = tf.float32))
        
}
b = tf.Variable(tf.random_normal([10]))

# Convolution operation
conv_z = tf.nn.conv2d(X_reshaped, weights["W_z"], strides = [1, 1, 1, 1], padding = "VALID")
conv_z_reshaped = tf.nn.tanh(tf.transpose(conv_z, perm = [0,2,3,1]))
conv_f = tf.nn.conv2d(X_reshaped, weights["W_f"], strides = [1, 1, 1, 1], padding = "VALID")
conv_f_reshaped = tf.nn.sigmoid(tf.transpose(conv_f, perm = [0,2,3,1]))
conv_o = tf.nn.conv2d(X_reshaped, weights["W_o"], strides = [1, 1, 1, 1], padding = "VALID")
conv_o_reshaped = tf.nn.sigmoid(tf.transpose(conv_o, perm = [0,2,3,1]))

# Pooling operation
with tf.device('/gpu:0'):
    for j in range(batch_size):
#         print j
        h = []
        c = []
        for i in range(weights["c"].get_shape()[0]):
#             print len(c)
            if i==0:
                c.append(((1-conv_f_reshaped)[j,i,:,:]*conv_z_reshaped[j,i,:,:]))
            else:
                c.append((c[i-1]*conv_f_reshaped[j,i,:,:]) + ((1-conv_f_reshaped)[j,i,:,:]*conv_z_reshaped[j,i,:,:]))

            h.append(c[i]*conv_o_reshaped[j,i,:,:])
        c_pack = tf.pack(c)
#         print c_pack.get_shape(),"rt"
        h_pack = tf.pack(h)
        weights["c"] = c_pack
#         print weights["c"].get_shape(),"rt1"
        weights["h"] = h_pack

fc1 = tf.reshape(weights["h"], [-1, 560])
# fc1 = tf.nn.tanh(tf.add(tf.matmul(fc1, weights["W"]), b))



In [11]:
# Number of timesteps(T in the paper) and no of filters (m in the paper) are 28 and 20 respectively
weights = {
    "W_z" : tf.Variable(tf.random_normal([28, 28, 1, 20])),
    "W_f" : tf.Variable(tf.random_normal([28, 28, 1, 20])),
    "W_o" : tf.Variable(tf.random_normal([28, 28, 1, 20])),
    "W" : tf.Variable(tf.random_normal([400, 10])),
    "h" : tf.Variable(tf.zeros([1, 28, 20, 1], dtype = tf.float32)),
    "c" : tf.Variable(tf.zeros([1, 28, 20, 1], dtype = tf.float32))
        
}
b = tf.Variable(tf.random_normal([10]))

In [13]:
pred = update(X_reshaped, weights, b)

ValueError: slice index 9 of dimension 1 out of bounds. for 'strided_slice_35' (op: 'StridedSlice') with input shapes: [128,9,20,28], [4], [4], [4].

In [39]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    c,b = mnist_data.train.next_batch(batch_size)
    d,e = sess.run([X_reshaped,conv_z_reshaped], feed_dict={X : c,Y: b})
    print d.shape, e.shape


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


KeyboardInterrupt: 